https://stackoverflow.com/questions/40590216/is-it-legal-to-index-into-a-struct

In [4]:
#include <iostream>
class A
{
    private:
    int a;

    public:
    A() : A(5){}
    A(int a)
    {
        this->a = a;
        std::cout << "yolo" << std::endl;
    }
    int getA(){ return a; }
};

In [5]:
{
    A a; //default constructor will be called
    std::cout << a.getA() << std::endl;
}

yolo
5


The default constructor is special because it is called when an object is declared but not initilaized with any arguments.   
Default constructor must always exist: When any constructor is explicitly declared in a class, no implicit default constructors is automatically provided. 

A a(); -> function prototybe, not actually creating an object  
A a;   -> default constructor called  (preferred)  
A a(5);  -> (preferred)  
A a = 5;  
A a {5};  
A a = {5};  

{} braces are called uniform initializer syntax  

https://stackoverflow.com/a/37228443  
Diff between using {} and ()  
() is more functional  
{} is for simply filling in parameter values: 

When an object is created, compiler makes sure that constructors for all of its subobjects (its member and inherited objects) are called.
https://www.geeksforgeeks.org/when-are-constructors-called/

In [1]:
class Rectangle {
  int width, height;
public:
  Rectangle(int x, int y) : width(x), height(y) {}
  int area(void) { return width * height; }
};

Rectangle* baz = new Rectangle[2] { {2,5}, {3,6} }; //universal initalizer

In [17]:
{
    class Rectangle {
        int width, height;
    public:
        Rectangle() : Rectangle(0, 0) {} //must specify a default constructor
        Rectangle(int x, int y) : width(x), height(y) {}
        int area(void) { return width * height; }
    };
    
    Rectangle* baz = new Rectangle[2]; //objects have already called default constructor
    //must use a member function to set values, or specify a copy constructor
//     baz[0] = Rectangle(2, 5); won't work
//     baz[1] = Rectangle(3, 6);
//     { {2,5}, {3,6} }; //universal initalizer
}

# Static Members

Must be initalized outside of class

In [ ]:
// static members in classes
#include <iostream>
using namespace std;

class Dummy {
  public:
    static int n;
    Dummy () { n++; };
};

int Dummy::n=0;

int main () {
  Dummy a;
  Dummy b[5];
  cout << a.n << '\n';
  Dummy * c = new Dummy;
  cout << Dummy::n << '\n';
  delete c;
  return 0;
}

# Const

Const objects can't be modified, obviously

In [ ]:
// constructor on const object
#include <iostream>
using namespace std;

class MyClass {
  public:
    int x;
    MyClass(int val) : x(val) {}
    int get() {return x;}
};

int main() {
  const MyClass foo(10);
// foo.x = 20;            // not valid: x cannot be modified
  cout << foo.x << '\n';  // ok: data member x can be read
  return 0;
}

Very good read:  
http://www.cplusplus.com/doc/tutorial/classes2/

Destructor, copy constructor, move constructor, etc. will all have an implicit one if not explicity defined

# Destructor

# Copy constructor

In [ ]:
MyClass::MyClass (const MyClass&);

Implicit copy constructor only does shallow copy of its member variables, if deep copy is needed, must specify:

In [ ]:
// Shallow copy ex: 
// What implicit basically does

class MyClass {
  public:
    int a, b; string c;
};

MyClass::MyClass(const MyClass& x) : a(x.a), b(x.b), c(x.c) {}

In [ ]:
// Deep copy ex:

#include <iostream>
#include <string>
using namespace std;

class Example5 {
    string* ptr;
  public:
    Example5 (const string& str) : ptr(new string(str)) {}
    ~Example5 () {delete ptr;}
    // copy constructor:
    Example5 (const Example5& x) : ptr(new string(x.content())) {}
    // access content:
    const string& content() const {return *ptr;}
};

int main () {
  Example5 foo ("Example");
  Example5 bar = foo;

  cout << "bar's content: " << bar.content() << '\n';
  return 0;
}

# Copy Assignment

In [ ]:
MyClass& operator= (const MyClass&);

Implicit copy assignment does a shallow copy

In [ ]:
Example5& operator= (const Example5& x) {
  delete ptr;                      // delete currently pointed string
  ptr = new string (x.content());  // allocate space for new string, and copy
  return *this;
}

What does the below do?

In [ ]:
Example5& operator= (const Example5& x) {
  *ptr = x.content();
  return *this;
}

# Move Constructor & Assignment

In [ ]:
MyClass fn();            // function returning a MyClass object
MyClass foo;             // default constructor
MyClass bar = foo;       // copy constructor
MyClass baz = fn();      // move constructor
foo = bar;               // copy assignment
baz = MyClass();         // move assignment 

The move assignment is called when an object is assigned the value of an unnamed object (an unnamed temporary).  
Default behavior is shallow copy.

Must read in this order:  
https://www.internalpointers.com/post/understanding-meaning-lvalues-and-rvalues-c  
https://www.internalpointers.com/post/c-rvalue-references-and-move-semantics-beginners  

https://en.cppreference.com/w/cpp/utility/move  
https://www.fluentcpp.com/2018/02/06/understanding-lvalues-rvalues-and-their-references/  

Now an lvalue reference is a reference that binds to an lvalue. lvalue references are marked with one ampersand (&).
And an rvalue reference is a reference that binds to an rvalue. rvalue references are marked with two ampersands (&&).

**lvalue reference vs pointer:**

A pointer is the address of the memory location. You can change the value of that address to point at different memory addresses.

A reference is an *alias* of the variable. You can only assign this alias during declaration. You cannot change which variable the reference is an alias of after it's declared.


Default to references unless dynamically allocating memory or setting pointer to another address.
Convention is also discussed below:
https://stackoverflow.com/a/21884880

`std::move` converts lvalue to rvalue reference, so you can pass it into move constructors (or asssignments) explicitly  
Otherwise, it will go default into 1) memory creation of the temporary, 2) the actual object copy-constructor operation  
^ which will be expensive  

Compiler will do RVO (return value optimization) for returning values, but not for input (like in use cases of std::move)  

When you std::move an lvalue, you need to be sure you won’t use it any more, because it will be considered like a disposable object by the rest of the code.

- Copying from A to B means that new memory is allocated to B and then the entire content of A is copied to this new memory allocated for B.
- Moving from A to B means that the memory already allocated to A is transferred to B without allocating any new storage. It involves simply copying the pointer (shallow copy).

In [7]:
{
    int a = 5;

    // lvalue reference: reference to lvalue (original lvalue can be modified)
    int& b = a;

    // rvalue reference: reference to rvalue (original rvalue can be modified)
    int&& c = 2;
}

In [ ]:
// move constructor/assignment
#include <iostream>
#include <string>
using namespace std;

class Example6 {
    string* ptr;
  public:
    Example6 (const string& str) : ptr(new string(str)) {}
    ~Example6 () {delete ptr;}
    // move constructor
    Example6 (Example6&& x) : ptr(x.ptr) {x.ptr=nullptr;}
    // move assignment
    Example6& operator= (Example6&& x) {
      delete ptr; //deletion is necessary since it's not a constructor, must delete any previous values
      ptr = x.ptr;
      x.ptr=nullptr;
      return *this;
    }
    // access content:
    const string& content() const {return *ptr;}
    // addition:
    Example6 operator+(const Example6& rhs) {
      return Example6(content()+rhs.content());
    }
};


int main () {
  Example6 foo ("Exam");
  Example6 bar = Example6("ple");   // move-construction
  
  foo = foo + bar;                  // move-assignment

  cout << "foo's content: " << foo.content() << '\n';
  return 0;
}

Compilers already optimize many cases that formally require a move-construction call in what is known as Return Value Optimization. Most notably, when the value returned by a function is used to initialize an object. In these cases, the move constructor may actually never get called.

Note that even though rvalue references can be used for the type of any function parameter, it is seldom useful for uses other than the move constructor. Rvalue references are tricky, and *unnecessary uses may be the source of errors quite difficult to track*.

# Implicit Members

# Friend Functions

https://stackoverflow.com/a/2315264

- 1. access to internals of class  
- 2. are in the scope of the class  
- 3. must be invoked on an instance  

friends get only 1.  
static functions get 1 and 2.  

Keyword used at declaration

In [ ]:
// friend functions
#include <iostream>
using namespace std;

class Rectangle {
    int width, height;
  public:
    Rectangle() {}
    Rectangle (int x, int y) : width(x), height(y) {}
    int area() {return width * height;}
    friend Rectangle duplicate (const Rectangle&); //can pass in any object, not necessarily const reference
};

Rectangle duplicate (const Rectangle& param)
{
  Rectangle res;
  res.width = param.width*2;
  res.height = param.height*2;
  return res;
}

int main () {
  Rectangle foo;
  Rectangle bar (2,3);
  foo = duplicate (bar);
  cout << foo.area() << '\n';
  return 0;
}

# Friend Classes

In [ ]:
// friend class
#include <iostream>
using namespace std;

class Square;

class Rectangle {
    int width, height;
  public:
    int area ()
      {return (width * height);}
    void convert (Square a);
};

class Square {
  friend class Rectangle;
  private:
    int side;
  public:
    Square (int a) : side(a) {}
};

void Rectangle::convert (Square a) {
  width = a.side;
  height = a.side;
}
  
int main () {
  Rectangle rect;
  Square sqr (4);
  rect.convert(sqr);
  cout << rect.area();
  return 0;
}

Rectangle is a friend. Rectangle has access to all of Square's stuff. Rectangle can do stuff with Square's stuff (not the other way around).

### http://www.cplusplus.com/doc/tutorial/inheritance/

This public keyword after the colon (:) denotes the most accessible level the members inherited from the class that follows it (in this case Polygon) will have from the derived class (in this case Rectangle). Since public is the most accessible level, by specifying this keyword the derived class will inherit all the members with the same levels they had in the base class.


Just use public

class Daughter: protected Mother;  

This would set protected as the less restrictive access level for the members of Daughter that it inherited from mother. That is, all members that were public in Mother would become protected in Daughter. Of course, this would not restrict Daughter from declaring its own public members. That less restrictive access level is only set for the members inherited from Mother.

If no access level is specified for the inheritance, the compiler assumes private for classes declared with keyword class and public for those declared with struct.

In principle, a publicly derived class inherits access to every member of a base class except:

- its constructors and its destructor
- its assignment operator members (operator=)
- its friends
- its private members

*Unless otherwise specified, the constructors of a derived class calls the default constructor of its base classes (i.e., the constructor taking no arguments).*

In [8]:
// constructors and derived classes
#include <iostream>
using namespace std;

class Mother {
  public:
    Mother ()
      { cout << "Mother: no parameters\n"; }
    Mother (int a)
      { cout << "Mother: int parameter\n"; }
};

class Daughter : public Mother {
  public:
    Daughter (int a)
      { cout << "Daughter: int parameter\n\n"; }
};

class Son : public Mother {
  public:
    Son (int a) : Mother (a)
      { cout << "Son: int parameter\n\n"; }
};

In [9]:
{
  Daughter kelly(0);
  Son bud(0);
}

Mother: no parameters
Daughter: int parameter

Mother: int parameter
Son: int parameter



Default constructor is called for Daughter   

# Multiple Inheritance

<table class="boxed">
<tr><th>Access</th><th><code>public</code></th><th><code>protected</code></th><th><code>private</code></th></tr>
<tr><td>members of the same class</td><td class="yes">yes</td><td class="yes">yes</td><td class="yes">yes</td></tr>
<tr><td>members of derived class</td><td class="yes">yes</td><td class="yes">yes</td><td class="no">no</td></tr>
<tr><td>not members</td><td class="yes">yes</td><td class="no">no</td><td class="no">no</td></tr>
</table>

https://stackoverflow.com/a/1372858

In [ ]:
class A 
{
public:
    int x;
protected:
    int y;
private:
    int z;
};

class B : public A
{
    // x is public
    // y is protected
    // z is not accessible from B
};

class C : protected A
{
    // x is protected
    // y is protected
    // z is not accessible from C
};

class D : private A    // 'private' is default for classes
{
    // x is private
    // y is private
    // z is not accessible from D
};

Basically, public -> protected -> private, if inheriting as x, then anything higher than it will decay into x

In [ ]:
// multiple inheritance
#include <iostream>
using namespace std;

class Polygon {
  protected:
    int width, height;
  public:
    Polygon (int a, int b) : width(a), height(b) {}
};

class Output {
  public:
    static void print (int i);
};

void Output::print (int i) {
  cout << i << '\n';
}

class Rectangle: public Polygon, public Output {
  public:
    Rectangle (int a, int b) : Polygon(a,b) {}
    int area ()
      { return width*height; }
};

class Triangle: public Polygon, public Output {
  public:
    Triangle (int a, int b) : Polygon(a,b) {}
    int area ()
      { return width*height/2; }
};
  
int main () {
  Rectangle rect (4,5);
  Triangle trgl (4,5);
  rect.print (rect.area());
  Triangle::print (trgl.area());
  return 0;
}

print is static

# Pointers to base class

In [ ]:
// pointers to base class
#include <iostream>
using namespace std;

class Polygon {
  protected:
    int width, height;
  public:
    void set_values (int a, int b)
      { width=a; height=b; }
};

class Rectangle: public Polygon {
  public:
    int area()
      { return width*height; }
};

class Triangle: public Polygon {
  public:
    int area()
      { return width*height/2; }
};

int main () {
  Rectangle rect;
  Triangle trgl;
  Polygon * ppoly1 = &rect;
  Polygon * ppoly2 = &trgl;
  ppoly1->set_values (4,5);
  ppoly2->set_values (4,5);
  cout << rect.area() << '\n';
  cout << trgl.area() << '\n';
  return 0;
}

# Virtual Methods

In [ ]:
// virtual members
#include <iostream>
using namespace std;

class Polygon {
  protected:
    int width, height;
  public:
    void set_values (int a, int b)
      { width=a; height=b; }
    virtual int area ()
      { return 0; }
};

class Rectangle: public Polygon {
  public:
    int area ()
      { return width * height; }
};

class Triangle: public Polygon {
  public:
    int area ()
      { return (width * height / 2); }
};

int main () {
  Rectangle rect;
  Triangle trgl;
  Polygon poly;
  Polygon * ppoly1 = &rect;
  Polygon * ppoly2 = &trgl;
  Polygon * ppoly3 = &poly;
  ppoly1->set_values (4,5);
  ppoly2->set_values (4,5);
  ppoly3->set_values (4,5);
  cout << ppoly1->area() << '\n';
  cout << ppoly2->area() << '\n';
  cout << ppoly3->area() << '\n';
  return 0;
}

The member function area has been declared as virtual in the base class because it is later redefined in each of the derived classes. Non-virtual members can also be redefined in derived classes, but **non-virtual members of derived classes cannot be accessed through a reference of the base class**: i.e., if virtual is removed from the declaration of area in the example above, all three calls to area would return zero, because in all cases, the version of the base class would have been called instead.

A class that declares or inherits a virtual function is called a polymorphic class.

https://stackoverflow.com/a/39932616

- Use **virtual** for the base class function declaration. 
- This is technically necessary.


- Use **override** (only) for a derived class' override.
- This helps maintenance.

In [ ]:
struct Base { virtual void foo() {} };
struct Derived: Base { void foo() override {} };

# Abstract Class

In [ ]:
// abstract class CPolygon
class Polygon {
  protected:
    int width, height;
  public:
    void set_values (int a, int b)
      { width=a; height=b; }
    virtual int area () = 0;
};

Abstract class with only pure virtual methods is an interface

In [ ]:
// pure virtual members can be called
// from the abstract base class
#include <iostream>
using namespace std;

class Polygon {
  protected:
    int width, height;
  public:
    void set_values (int a, int b)
      { width=a; height=b; }
    virtual int area() =0;
    void printarea()
      { cout << this->area() << '\n'; }
};

class Rectangle: public Polygon {
  public:
    int area (void)
      { return (width * height); }
};

class Triangle: public Polygon {
  public:
    int area (void)
      { return (width * height / 2); }
};

int main () {
  Rectangle rect;
  Triangle trgl;
  Polygon* ppoly1 = &rect;
  Polygon* ppoly2 = &trgl;
  ppoly1->set_values (4,5);
  ppoly2->set_values (4,5);
  ppoly1->printarea();
  ppoly2->printarea();
  return 0;
}

In [ ]:
// dynamic allocation and polymorphism
#include <iostream>
using namespace std;

class Polygon {
  protected:
    int width, height;
  public:
    Polygon (int a, int b) : width(a), height(b) {}
    virtual int area (void) =0;
    void printarea()
      { cout << this->area() << '\n'; }
};

class Rectangle: public Polygon {
  public:
    Rectangle(int a,int b) : Polygon(a,b) {}
    int area()
      { return width*height; }
};

class Triangle: public Polygon {
  public:
    Triangle(int a,int b) : Polygon(a,b) {}
    int area()
      { return width*height/2; }
};

int main () {
  Polygon * ppoly1 = new Rectangle (4,5);
  Polygon * ppoly2 = new Triangle (4,5);
  ppoly1->printarea();
  ppoly2->printarea();
  delete ppoly1;
  delete ppoly2;
  return 0;
}